In [ ]:
import pandas as pd
import requests
import time
import os

# ====== SETUP ======
OUTPUT_FILE = "edges_with_distance_duration_osrm_231.csv"
OSRM_URL = "http://localhost:5000"  # asumsi osrm-routed sudah jalan di sini

# ====== STEP 1: Load SPKLU file ======
df = pd.read_csv("long_lat_spklu_sumatera.csv")
id_to_coord = {i: (row['Longitude'], row['Latitude']) for i, row in df.iterrows()}
nodes = list(id_to_coord.keys())

# ====== STEP 2: Load progress (resume jika file sudah ada) ======
if os.path.exists(OUTPUT_FILE):
    cached_df = pd.read_csv(OUTPUT_FILE)
    processed_pairs = set(zip(cached_df["from_node"], cached_df["to_node"]))
    results = cached_df.to_dict("records")
    print(f"🔁 Melanjutkan dari hasil sebelumnya: {len(processed_pairs)} edge sudah diproses.")
else:
    results = []
    processed_pairs = set()

total_pairs = 0
successful_pairs = len(processed_pairs)

# ====== STEP 3: Loop semua pasangan (from_node, to_node) ======
for from_id in nodes:
    lon1, lat1 = id_to_coord[from_id]

    for to_id in nodes:
        if from_id == to_id or (from_id, to_id) in processed_pairs:
            continue

        lon2, lat2 = id_to_coord[to_id]
        total_pairs += 1

        try:
            url = f"{OSRM_URL}/route/v1/driving/{lon1},{lat1};{lon2},{lat2}?overview=false"
            response = requests.get(url)
            data = response.json()

            if data["code"] == "Ok":
                route = data["routes"][0]
                distance_km = round(route["distance"] / 1000, 2)
                duration_min = round(route["duration"] / 60, 2)

                edge = {
                    "from_node": from_id,
                    "to_node": to_id,
                    "from_name": df.iloc[from_id]["Nama SPKLU"],
                    "to_name": df.iloc[to_id]["Nama SPKLU"],
                    "distance_km": distance_km,
                    "duration_min": duration_min
                }
                results.append(edge)
                processed_pairs.add((from_id, to_id))
                successful_pairs += 1
                print(f"✅ {from_id} → {to_id}: {distance_km} km, {duration_min} min")
            else:
                print(f"❌ {from_id} → {to_id} gagal: {data['code']}")

        except Exception as e:
            print(f"🔥 Error: {e} → {from_id} → {to_id}, skip.")

        # ✅ SELALU simpan hasil sementara ke CSV
        pd.DataFrame(results).to_csv(OUTPUT_FILE, index=False)

        time.sleep(0.2)  # throttle ringan agar tidak overload

print(f"\n✅ Selesai. Total pairs diperiksa: {total_pairs}, berhasil: {successful_pairs}")
print(f"📁 File hasil: {OUTPUT_FILE}")


In [ ]:
import pandas as pd
import networkx as nx
import pickle

# Load file koordinat SPKLU (node data)
node_df = pd.read_csv("long_lat_spklu_sumatera.csv")

# Load edge data (hasil dari Google Maps API)
edge_df = pd.read_csv("edges_with_distance_duration_osrm_231.csv")

# Buat dictionary node: index -> atribut
node_attributes = {
    i: {
        "name": row["Nama SPKLU"],
        "latitude": row["Latitude"],
        "longitude": row["Longitude"]
    }
    for i, row in node_df.iterrows()
}

# Buat directed graph
G = nx.DiGraph()

# Tambahkan node ke graph
for node_id, attrs in node_attributes.items():
    G.add_node(node_id, **attrs)

# Tambahkan edge ke graph (hilangkan self-loop)
for _, row in edge_df.iterrows():
    from_id = row["from_node"]
    to_id = row["to_node"]
    if from_id != to_id:
        G.add_edge(
            from_id,
            to_id,
            weight=row["duration_min"],
            distance=row["distance_km"]
        )

# Simpan graph ke format pickle
with open("spklu_sumatera_graph_fully_connected_231.pkl", "wb") as f:
    pickle.dump(G, f)


In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import pickle

# ====== STEP 5: Visualisasi ======
# Posisi node berdasarkan longitude & latitude
pos = {
    node: (data["longitude"], data["latitude"])
    for node, data in G.nodes(data=True)
}

plt.figure(figsize=(16, 12))

# ⚠️ arrows=False agar label bisa ditampilkan tanpa error
nx.draw_networkx_edges(G, pos, alpha=0.2, arrows=False)
nx.draw_networkx_nodes(G, pos, node_size=80, node_color="orange")

node_labels = {node: data["name"] for node, data in G.nodes(data=True)}
nx.draw_networkx_labels(G, pos, labels=node_labels, font_size=6)

edge_labels = {
    (u, v): f'{d["weight"]:.1f}m' for u, v, d in G.edges(data=True)
}
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=5)

plt.title("Visualisasi Graph SPKLU Sumatera\n(Node: Nama SPKLU, Edge: Durasi Tempuh [menit])")
plt.axis("off")
plt.tight_layout()
plt.show()

In [ ]:
import pandas as pd
import networkx as nx
import pickle
import ast
import random

# ===== STEP 1: Load CSV files =====
node_df = pd.read_csv("long_lat_spklu_sumatera.csv")
edge_df = pd.read_csv("edges_with_distance_duration_osrm_231.csv")
parameter_df = pd.read_csv("ketersediaan_history_with_parameters.csv")

# ===== STEP 2: Buat dictionary node =====
node_attributes = {
    i: {
        "name": row["Nama SPKLU"],
        "latitude": row["Latitude"],
        "longitude": row["Longitude"]
    }
    for i, row in node_df.iterrows()
}

# ===== STEP 3: Inisialisasi graph directed =====
G = nx.DiGraph()

# Tambahkan node
for node_id, attrs in node_attributes.items():
    attrs.update({
        'is_charging_station': True,
        # 'total_slots': 2,
        # 'slots_charging_rate': [1.0, 2.0],
        # 'slots_availability_std': 0.5,
        # 'slots_availability_mean': 1.55,
        # 'slots_availability': [True, True]
    })
    G.add_node(node_id, **attrs)

# ===== STEP 4: Tambahkan edge =====
for _, row in edge_df.iterrows():
    from_id = row["from_node"]
    to_id = row["to_node"]
    if from_id != to_id:
        weight = max(row["duration_min"], 0.000001)
        distance = max(row["distance_km"], 0.000001)
        G.add_edge(
            from_id,
            to_id,
            weight=weight,
            distance=distance,
            mean=weight,
            std=weight * 0.1
        )

# ===== STEP 5: Tambahkan atribut parameter dan indikator =====
name_to_node = {data["name"]: node_id for node_id, data in G.nodes(data=True)}

for _, row in parameter_df.iterrows():
    name = row["Nama SPKLU"]
    if name in name_to_node:
        node_id = name_to_node[name]
        try:
            # Konversi kolom Parameter dari string ke dict
            parameter = ast.literal_eval(row["Parameter"])
            indicator = {}
            for rate, param in parameter.items():
                s = param.get("s", 1)
                p = param.get("p", 0.025)
                # Hitung jumlah occupied slot berdasarkan probabilitas p
                occupied = sum([1 if random.random() < p else 0 for _ in range(s)])
                indicator[rate] = round(occupied / s, 2)
            # Tambahkan ke node
            G.nodes[node_id]["slots_parameter"] = parameter
            G.nodes[node_id]["slots_indicator"] = indicator
        except Exception as e:
            print(f"⚠️ Error parsing parameter untuk {name}: {e}")

# ===== STEP 6: Simpan graph =====
with open("spklu_sumatera_graph_with_parameters_231.pkl", "wb") as f:
    pickle.dump(G, f)

print("✅ Graph berhasil disimpan ke spklu_sumatera_graph_with_parameters_231.pkl")

# ===== STEP 7: Contoh output =====
print("\n📍 Atribut 5 node pertama:")
for node_id, data in list(G.nodes(data=True))[:5]:
    print(f"Node {node_id}: {data}")

print("\n🔗 Atribut 5 edge pertama:")
for from_id, to_id, data in list(G.edges(data=True))[:5]:
    print(f"Edge {from_id} → {to_id}: {data}")

In [ ]:
import pickle
import networkx as nx

# Load graph dari file pickle
with open("spklu_sumatera_graph_with_parameters_231.pkl", "rb") as f:
    G = pickle.load(f)

# Tampilkan 5 node pertama dan atributnya
print("Lima node pertama dan atributnya:")
for node, attr in list(G.nodes(data=True))[:5]:
    print(f"Node ID: {node}")
    for key, value in attr.items():
        print(f"  {key}: {value}")
    print("-" * 40)
